In [1]:
from selenium_crawl import SeleniumParser

In [4]:
sp = SeleniumParser(visible=True,proxy=True,proxy_ip = '192.168.3.57',proxy_port=24000)
soup = sp.get_soup('https://myip.ht/')

In [5]:
sp.exit()